In [2]:
import torch
import os
from ultralytics import YOLO
import cv2
import numpy as np

In [13]:
# model = YOLO("../models/xpm_annotate.pt")
model = YOLO("../models/yolov8s.pt")
# model = YOLO("../models/clahe_8vs.pt")
# model = YOLO("../models/roboflow_8vs.pt")

## Individual Image

In [25]:
img_path = 'dataset/raw/sample1.jpg'
output_dir = os.path.join('yolo_predictions/yolov8s', os.path.basename(img_path).split('.')[0])
og_filename = os.path.basename(img_path) 
os.makedirs('yolo_predictions/yolov8s', exist_ok=True)

In [20]:
results = model.predict(source=img_path, conf=0.5, save=False, save_txt=False)
masks = results[0].masks
boxes = results[0].boxes


image 1/1 c:\Users\kimbe\Documents\GitHub\mandible-height-estimation\dataset\raw\sample1.jpg: 288x640 1 mandible, 155.6ms
Speed: 2.0ms preprocess, 155.6ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 640)


In [21]:
input_img = cv2.imread(img_path)
input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)

In [22]:
mask_color = (250, 200, 0)

In [28]:
output_dir = "yolo_predictions/yolov8s"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for idx, mask in enumerate(masks):
    m = torch.squeeze(mask.data)

    mask_np = m.cpu().numpy().astype(np.uint8)

    colored_mask = np.zeros((mask_np.shape[0], mask_np.shape[1], 3), dtype=np.uint8)
    colored_mask[mask_np == 1] = mask_color

    colored_mask = cv2.resize(colored_mask, (input_img.shape[1], input_img.shape[0]))

    alpha = 0.5  
    overlayed_img = cv2.addWeighted(input_img, 1, colored_mask, alpha, 0)

    new_name = og_filename.replace('.jpg', '') + '_overlay' + '.jpg'

    cv2.imwrite(os.path.join(output_dir, new_name), cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))

    cv2.imshow('Image', cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [36]:
input_dir = "dataset/box_jpeg"
output_dir = "yolo_predictions/yolov8s/box"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        input_img = cv2.imread(os.path.join(input_dir, filename))

        for idx, mask in enumerate(masks):
            m = torch.squeeze(mask.data)

            mask_np = m.cpu().numpy().astype(np.uint8)

            colored_mask = np.zeros((mask_np.shape[0], mask_np.shape[1], 3), dtype=np.uint8)
            colored_mask[mask_np == 1] = mask_color

            colored_mask = cv2.resize(colored_mask, (input_img.shape[1], input_img.shape[0]))

            alpha = 0.5  
            overlayed_img = cv2.addWeighted(input_img, 1, colored_mask, alpha, 0)

            new_name = filename.replace('.jpg', '') + '_overlay' + '.jpg'

            cv2.imwrite(os.path.join(output_dir, new_name), cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))

## One Directory

In [14]:
input_dir = "../dataset/box/teeth_nopins"
output_dir = "../compare/xpm"

In [15]:
image_files = os.listdir(input_dir)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

mask_color = (250, 200, 0)

for image_file in image_files:
    img_path = os.path.join(input_dir, image_file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    filename = os.path.basename(img_path) 

    results = model.predict(source=img_path, conf=0.5, save=False, save_txt=False)
    masks = results[0].masks

    if masks is not None:  
        boxes = results[0].boxes

        for mask in masks:
            if mask is not None:
                m = torch.squeeze(mask.data)

                mask_np = m.cpu().numpy().astype(np.uint8)

                colored_mask = np.zeros((mask_np.shape[0], mask_np.shape[1], 3), dtype=np.uint8)
                colored_mask[mask_np == 1] = mask_color

                colored_mask = cv2.resize(colored_mask, (img.shape[1], img.shape[0]))

                alpha = 0.5  
                overlayed_img = cv2.addWeighted(img, 1, colored_mask, alpha, 0)

                new_name = filename.replace('.jpg', '') + '_overlay' + '.jpg'

                cv2.imwrite(os.path.join(output_dir, new_name), cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))


image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\dataset\box\teeth_nopins\0304958.tiff: 320x640 1 mandible, 118.2ms
Speed: 2.0ms preprocess, 118.2ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\dataset\box\teeth_nopins\0405342.tiff: 320x640 1 mandible, 109.2ms
Speed: 1.0ms preprocess, 109.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\dataset\box\teeth_nopins\0413594-1.tiff: 320x640 1 mandible, 142.6ms
Speed: 3.0ms preprocess, 142.6ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\dataset\box\teeth_nopins\0810504-2.tiff: 320x640 1 mandible, 121.0ms
Speed: 2.0ms preprocess, 121.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\dataset\box\teeth_nop